In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 14.6 ms


In [2]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'

final_pt_df2 = pd.read_csv(wd + '/data/raw/csv/16082018_final_pt_df2.csv' , index_col=0)
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 94.8 ms


In [3]:
#reading in all of my data that is not limited to 72 hour time window between t_0 and t+72
date= '27082018' 

vaso_dose_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_vaso_dose_72.csv' %(date), index_col=0)
#ventsettings_72=pd.read_csv(
#    wd+'/data/raw/csv/72_hr_window/%s_ventsettings_72.csv' %(date), index_col=0)

ventcategory_df=pd.read_csv(
    wd+'/data/raw/csv/%s_ventcategory_df.csv' %(date), index_col=0)

echodata_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_echodata_72.csv' %(date), index_col=0)
labs_all_nosummary_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_labs_all_nosummary_72.csv' %(date), index_col=0)


weightfirstday_df=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_weightfirstday_df.csv' %(date), index_col=0)
heightfirstday_df=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_heightfirstday_df.csv' %(date), index_col=0)

vitals_all_nosummary_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_vitals_all_nosummary_72.csv' %(date), index_col=0)
uti_all_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_uti_all_72.csv' %(date), index_col=0)
bg_all_nosummary_72=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_bg_all_nosummary_72.csv' %(date), index_col=0)

rrt_merged_allpt_df=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_rrt_merged_allpt_df.csv' %(date), index_col=0)
gcs72_df=pd.read_csv(
    wd+'/data/raw/csv/72_hr_window/%s_gcs72_df.csv' %(date), index_col=0)

sofa_df_72=pd.read_csv(
    wd+'/data/raw/csv/%s_sofa_df_72.csv' %(date), index_col=0)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 14.4 s


In [4]:
missingness_df=pd.read_csv(
    wd+'/data/interim/%s_missingness_df.csv' %(date), index_col=0)

time: 4.77 ms


### from nelson's email after seeing missingness:
#This looks good. As expected, there are some patents (something like 3%) who don't have even some basic vital signs.
#As a pragmatic cutoff, I suggest that we take off patients who don't have at least an SpO2 reading (approx 3.5% of patients),
#I think that is a reasonable minimum of data to have. 
#I will be looking at this more carefully in the coming week or so.

# goal: refine cohort to those with bare minimum vitals

In [61]:
missingness_df.loc[missingness_df.loc[:,'label']=='SpO2',:]

,label,%missingness,source,data_label
5,SpO2,3.54442,vitals_all_nosummary_72,label


time: 8.39 ms


#   goal: explore the 12-pandas-techniqus-python-data-manipulation 

In [ ]:
#   goal: explore the 12-pandas-techniqus-python-data-manipulation 

### pivot tables

In [55]:
from scipy.stats import mode

sofa_pivot= sofa_df_72.pivot_table(values=['sofa'], index=['subject_id','day'], aggfunc=np.sum)
#values are on a different index than the two indexes, hierarchical idex. 
sofa_pivot

sofa
subject_id day      
9          4.0     6
           5.0     7
19         1.0     0
21         6.0     8
           7.0     9
           8.0     5
25         0.0     4
           1.0     1
           2.0     2
31         7.0     7
33         1.0     1
36         5.0     1
           6.0     1
61         2.0     4
68         4.0     7
           5.0     2
           6.0     4
73         1.0     2
           2.0     1
80         1.0     0
94         1.0     0
105        0.0     3
           1.0     5
           2.0     1
           3.0     1
106        6.0     0
109        0.0    12
           1.0    17
           2.0     6
           3.0     5
...              ...
99863      3.0     4
           4.0     4
99873      1.0     5
99881      1.0     8
           2.0     5
99883      0.0     8
           1.0     1
99894      1.0     3
           2.0     3
           3.0     1
99901      6.0     1
99912      1.0     7
           2.0     5
           3.0     6
99913      0.0     4
           1.0     5
           2.0     3
99923      0.0     8
           1.0     4
           2.0     4
99934      2.0     1
99955      1.0     2
           2.0     5
           3.0     3
99957      1.0     0
           2.0     1
99966      1.0     3
99973      2.0     1
99995      1.0     2
           2.0     1

[28561 rows x 1 columns]

time: 35.1 ms


In [56]:
sofa_pivot['sofa']

subject_id  day
9           4.0     6
            5.0     7
19          1.0     0
21          6.0     8
            7.0     9
            8.0     5
25          0.0     4
            1.0     1
            2.0     2
31          7.0     7
33          1.0     1
36          5.0     1
            6.0     1
61          2.0     4
68          4.0     7
            5.0     2
            6.0     4
73          1.0     2
            2.0     1
80          1.0     0
94          1.0     0
105         0.0     3
            1.0     5
            2.0     1
            3.0     1
106         6.0     0
109         0.0    12
            1.0    17
            2.0     6
            3.0     5
                   ..
99863       3.0     4
            4.0     4
99873       1.0     5
99881       1.0     8
            2.0     5
99883       0.0     8
            1.0     1
99894       1.0     3
            2.0     3
            3.0     1
99901       6.0     1
99912       1.0     7
            2.0     5
            3.0 

time: 8.8 ms


In [25]:
scipy.stats?

Object `scipy.stats` not found.
time: 1.31 ms


In [48]:
arrays =[['bar','bar','baz','baz','foo','foo','qux','qux'],
        ['one','two','one','two','one','two','one','two']]
tuples = list(zip(*arrays))
index =pd.MultiIndex.from_tuples(tuples, names=['name1','name2']) #forming a multiindex
#pd.DataFrame(np.random.randn(3,8), index=['A','B','C'], columns=index)
s= pd.Series(np.random.randn(8),index=index) #populating multiindex with random numbers
s

name1  name2
bar    one      1.647952
       two     -0.679173
baz    one      1.244170
       two      1.288899
foo    one      0.129095
       two      0.580954
qux    one     -0.582726
       two     -0.063222
dtype: float64

time: 9.62 ms


In [50]:
df= pd.DataFrame(np.random.randn(3,8), index=['A','B','C'], columns=index) 

time: 1.46 ms
